In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import time

from selenium import webdriver

In [2]:
years = list(range(1991, 2023))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

# Scrape MVPs


In [3]:
dfs = []

# Already fetched
# for year in years:
#   url = url_start.format(year)
#   data = requests.get(url)
#   file_name = "./mvps/{}.html".format(year)


#   if os.path.exists(file_name):
#     os.remove(file_name)
#   with open(file_name, "w+") as f:
#     f.write(data.text)

In [4]:
for year in years:
    with open("./mvps/{}.html".format(year)) as f:
        page = f.read()

    soup = bs(page, "html.parser")
    soup.find("tr", class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year

    print(year)

    dfs.append(mvp)
    mvps = pd.concat(dfs)

/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


1991


/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


1992
1993


/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


...

/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


2018
2019


/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


2020
2021
2022


/tmp/ipykernel_563/2332508481.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


In [5]:
if os.path.exists("mvps.csv"):
    os.remove("mvps.csv")
mvps.to_csv("mvps.csv")

# Scrape player stats


In [6]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [7]:
# options = webdriver.FirefoxOptions()
# driver = webdriver.Remote(command_executor=os.environ["SELENIUM_URL"], options=options)

# for year in years:
#     url = player_stats_url.format(year)
#     print(f"scraping {year} data")

#     driver.get(url)
#     driver.execute_script("window.scrollTo(1,10000)")
#     time.sleep(2)

#     with open("player/{}.html".format(year), "w+") as f:
#         f.write(driver.page_source)

# driver.close()

scraping 1991 data
scraping 1992 data
scraping 1993 data
scraping 1994 data
scraping 1995 data
scraping 1996 data
scraping 1997 data
scraping 1998 data
scraping 1999 data
scraping 2000 data
scraping 2001 data
scraping 2002 data
scraping 2003 data
scraping 2004 data
scraping 2005 data
scraping 2006 data
scraping 2007 data
scraping 2008 data
scraping 2009 data
scraping 2010 data
scraping 2011 data
scraping 2012 data
scraping 2013 data
scraping 2014 data
scraping 2015 data
scraping 2016 data
scraping 2017 data
scraping 2018 data
scraping 2019 data
scraping 2020 data
scraping 2021 data
scraping 2022 data


In [8]:
dfs = []

for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()

    soup = bs(page, "html.parser")
    soup.find("tr", class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

players = pd.concat(dfs)
players.to_csv("players.csv")

/tmp/ipykernel_563/1847893316.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
/tmp/ipykernel_563/1847893316.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
/tmp/ipykernel_563/1847893316.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
/tmp/ipykernel_563/1847893316.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(st


# Team Stats

In [9]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [11]:
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    time.sleep(2)
    
    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [12]:
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
    
    soup = bs(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    dfs.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dfs.append(w_df)

/tmp/ipykernel_563/1228026030.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  e_df = pd.read_html(str(e_table))[0]
/tmp/ipykernel_563/1228026030.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  w_df = pd.read_html(str(w_table))[0]
/tmp/ipykernel_563/1228026030.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  e_df = pd.read_html(str(e_table))[0]
/tmp/ipykernel_563/1228026030.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  w_df = pd.read_html(str(w_table))[0]
/tmp/ipykernel_563/122

In [13]:
teams = pd.concat(dfs)
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,56,26,.683,—,115.6,109.9,5.37,2022,Memphis Grizzlies*
14,52,30,.634,4.0,108.0,104.7,3.12,2022,Dallas Mavericks*
15,36,46,.439,20.0,109.3,110.3,-0.84,2022,New Orleans Pelicans*
16,34,48,.415,22.0,113.2,113.0,0.02,2022,San Antonio Spurs


In [ ]:
teams.to_csv("teams.csv")